In [36]:
# =============================================================================
# Imports
# =============================================================================
import os
import sys
import json
import csv


In [37]:
# =============================================================================
# Declares
# =============================================================================
# Empirical 13-datasets
# aBSREL
aBSREL_DIR = "/Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL"
aBSREL_MH_DIR = "/Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL-MH"
aBSREL_OUTPUT_CSV = "PROCESSED_PairedAnalysis_13datasets_aBSREL-MH_vs_aBSREL.csv"


In [38]:
def read_json_aBSREL(filename):
    print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        return 9999991 
    # end if
    
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    
    
    # aBSREL, what do I want to grab?
    # All of the nodes, their p-value, LRT value, uncorrected p-value
    # Full adaptive model, AI C-c, LL, 
    # Baseline MG94xREV
    
    # --- Fits ----
    MG94_AICc = json_data["fits"]["Baseline MG94xREV"]["AIC-c"]
    MG94_LL = json_data["fits"]["Baseline MG94xREV"]["Log Likelihood"]
    FULL_AICc = json_data["fits"]["Full adaptive model"]["AIC-c"]
    FULL_LL = json_data["fits"]["Full adaptive model"]["Log Likelihood"]
    
    return MG94_AICc, MG94_LL, FULL_AICc, FULL_LL
#end method

def read_json_aBSREL_MH(filename_MH):
    print("# Reading:", filename_MH)
    if os.stat(filename_MH).st_size == 0: return 9999990
    with open(filename_MH, "r") as fh:
        json_data = json.load(fh)
    fh.close()

    # --- Fits ----
    MG94_MH_AICc = json_data["fits"]["Baseline MG94xREV"]["AIC-c"]
    MG94_MH_LL = json_data["fits"]["Baseline MG94xREV"]["Log Likelihood"]
    FULL_MH_AICc = json_data["fits"]["Full adaptive model"]["AIC-c"]
    FULL_MH_LL = json_data["fits"]["Full adaptive model"]["Log Likelihood"]
    
    # Baseline MG94xREV -- per branch distributions -- per branch delta,psi,omega
    return MG94_MH_AICc, MG94_MH_LL, FULL_MH_AICc, FULL_MH_LL
#end method

def clear_output(filename, column_headers):
    #Empty out the output CSV
    with open(filename, 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(column_headers)
    csvfile.close()
    #end with
#end method

def process_aBSREL(filename_MH, filename):
    global aBSREL_OUTPUT_CSV
    
    MG94_AICc, MG94_LL, FULL_AICc, FULL_LL = read_json_aBSREL(filename)
    MG94_MH_AICc, MG94_MH_LL, FULL_MH_AICc, FULL_MH_LL = read_json_aBSREL_MH(filename_MH)

    delta_AICc = float(FULL_MH_AICc) - float(FULL_AICc)
    delta_LL = float(FULL_MH_LL) - float(FULL_LL)
    
    #Output to csv
    with open(aBSREL_OUTPUT_CSV, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow([filename_MH.split("/")[-1].replace(".aBSREL.json", ""), 
                             FULL_AICc, FULL_LL,
                             MG94_MH_AICc, MG94_MH_LL, 
                             FULL_MH_AICc, FULL_MH_LL,
                             delta_AICc, delta_LL])
    #end with
    csvfile.close()
    #end with
#end method

In [32]:
# =============================================================================
# Main subroutine
# =============================================================================
print("# Init ")
print("# Program is in:", os.getcwd())
print("# Saving output to:", aBSREL_OUTPUT_CSV)
print("# Processing ", aBSREL_DIR)
print("# Processing ", aBSREL_MH_DIR)

column_headers = ["Filename", 
                      "aBSREL_FULL_AICc", "aBSREL_FULL_LL", 
                      "aBSREL-MH_MG94_AICc","aBSREL-MH_MG94_LL",
                      "aBSREL-MH_FULL_AICc", "aBSREL-MH_FULL_LL",
                      "delta_AICc(Full-MHversusFull)","delta_LL(Full-MHversusFull)"
                     ]

clear_output(aBSREL_OUTPUT_CSV, column_headers)

BASE_tag = ""
MH_tag = ""
aBSREL_DIR_FILES = [os.path.join(aBSREL_DIR, file.name) for file in os.scandir(aBSREL_DIR) if file.name.endswith(".aBSREL.json")]
aBSREL_MH_DIR_FILES = [os.path.join(aBSREL_MH_DIR, file.name) for file in os.scandir(aBSREL_MH_DIR) if file.name.endswith(".aBSREL-MH.json")]

print("# Number of aBSREL results:", len(aBSREL_DIR_FILES))
print("# Number of aBSREL-MH results:", len(aBSREL_MH_DIR_FILES))

# Loop over files in aBSREL-MH_DIR
matches_count = 0
for n, file in enumerate(aBSREL_MH_DIR_FILES):
    if os.stat(file).st_size == 0: 
        print("# Encountered empty file:", file)
        continue
    #end if
    
    #print("# Searching for match for:", file.split("/")[-1])
    print("#", n+1, "Searching for match for:", file)
    
    NEXUS_FILENAME = file.split("/")[-1].replace(".aBSREL-MH.json", "") 
    
    aBSREL_VERSION = os.path.join(aBSREL_DIR, NEXUS_FILENAME) + ".aBSREL.json"
    
    if os.stat(file).st_size > 0: 
        print("# Checking for aBSREL output for:", aBSREL_VERSION)
    else:
        continue
    #end if
    
    # Does it match with a file in aBSREL_DIR?, if soo..
    if aBSREL_VERSION in aBSREL_DIR_FILES:
        # process
        matches_count += 1
        process_aBSREL(file, aBSREL_VERSION)
    else:
        # dont process, but record this.
        pass
    # end if
    print()
#end for

print("# Number of matches:", matches_count)
             


# Init 
# Program is in: /Users/user/Documents/BS-MH/scripts
# Saving output to: PROCESSED_PairedAnalysis_13datasets_aBSREL-MH_vs_aBSREL.csv
# Processing  /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL
# Processing  /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL-MH
# Number of aBSREL results: 13
# Number of aBSREL-MH results: 13
# 1 Searching for match for: /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL-MH/ENCenv.nex.aBSREL-MH.json
# Checking for aBSREL output for: /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL/ENCenv.nex.aBSREL.json
# Reading: /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL/ENCenv.nex.aBSREL.json
# Reading: /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL-MH/ENCenv.nex.aBSREL-MH.json

# 2 Searching for match for: /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL-MH/HepatitisD.nex.aBSREL-MH.json
# Checking for aBSREL output for: /Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL/HepatitisD.nex.aBSREL.json
# 

In [9]:
# =============================================================================
# End of file
# =============================================================================